In [1]:
import matplotlib.pyplot as plt
import numpy as np
import qutip_qip
from qutip import (Qobj, SolverOptions, basis, fock_dm, qeye, sigmax, sigmay,
                   sigmaz, tensor)
from qutip.ipynbtools import version_table
from qutip_qip.circuit import QubitCircuit
from qutip_qip.compiler import GateCompiler, Instruction
from qutip_qip.device import Model, ModelProcessor
from qutip_qip.noise import Noise
from qutip_qip.operations import Gate
from scipy.optimize import curve_fit

In [2]:
class ClassicalCrossTalk(Noise):
    def __init__(self, ratio):
        self.ratio = ratio
    
    def get_noisy_dynamics(self, dims=None, pulses=None, systematic_noise=None):

        for i, pulse in eneumerate(pulses):
            if "sx" not in pulse.label and "sy" not in pulse.label:
                continue
            target = pulse.targets[0]
            if target != 0:
                pulses[i].add_control_noise(self.ratio * pulse.qobj,
                    targets=[target - 1],
                    coeff=pulse.coeff,
                    tlist=pulse.tlist,)
            if target != len(dims) - 1:  # add pulse to the right neighbour
                pulses[i].add_control_noise(
                    self.ratio * pulse.qobj,
                    targets=[target + 1],
                    coeff=pulse.coeff,
                    tlist=pulse.tlist,
                )
        return pulses, systematic_noise